In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
# from google.colab import drive

# drive.mount("/content/drive")


d:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_json(path):
    data = []
    with open(path, "r", encoding="utf-8") as fp:
        for line in fp:
            data.append(json.loads(line))
    return data


In [3]:
class BaselineData(Dataset):
    def __init__(self, data, tokenizer, config):
        self.data = data
        self.tokenizer = tokenizer
        self.pad_size = config.pad_size
        self.label2id = config.label2id

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = "[SEP]".join(
            [self.data[idx]["hashtag"]] + self.data[idx]["comments"]
        )

        input_ids, attention_mask = self.__convert_to_id__(sentence)

        if self.data[idx].get("attitudes"):
            label = self.__convert_label__(self.data[idx]["attitudes"])
            return (
                torch.tensor(input_ids),
                torch.tensor(attention_mask),
                torch.tensor(label),
            )
        else:
            return (
                torch.tensor(input_ids),
                torch.tensor(attention_mask),
            )

    def __convert_to_id__(self, sentence):
        ids = self.tokenizer.encode_plus(sentence)
        input_ids = self.__padding__(ids["input_ids"])
        attention_mask = self.__padding__(ids["attention_mask"])

        return input_ids, attention_mask

    def __convert_label__(self, label):
        onehot_label = [0] * 24
        for i in label:
            onehot_label[self.label2id[i]] = 1
        return onehot_label

    def __padding__(self, sentence):
        sentence = sentence[: self.pad_size]  # 长就截断
        sentence = sentence + [0] * (self.pad_size - len(sentence))  # 短就补充
        return sentence


In [4]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.bert = RobertaModel.from_pretrained(config.PTM)
        self.bert_config = RobertaConfig.from_pretrained(config.PTM)
        self.fc = nn.Linear(self.bert_config.hidden_size, self.bert_config.hidden_size)
        self.fc1 = nn.Linear(self.bert_config.hidden_size, config.label_num)
        self.act = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.bert(input_ids=x[0], attention_mask=x[1]).pooler_output
        x = self.sigmoid(self.fc1(self.act(self.fc(x))))
        return x


In [5]:
def train(config, dataset, model, optimizer, valid_dataset):
    max_scores = 0
    for epoch in range(config.epochs):
        with tqdm(total=len(dataset)) as pbar:
            for idx, data in enumerate(dataset):
                x = [data[0].long(), data[1].long()]
                y = data[2].float()
                y_hat = model(x)
                loss = F.binary_cross_entropy(y_hat, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.set_postfix({"loss": "{:.4f}".format(loss)})
                pbar.update(1)
        scores = valid(config, valid_dataset, model)
        if scores >= max_scores:
            max_scores = scores
            saved_model = model
    return saved_model


def valid(config, dataset, model):
    true = []
    pred = []
    with torch.no_grad():
        for idx, data in enumerate(dataset):
            x = [data[0].long(), data[1].long()]
            y = data[2].float().view(-1, 24).tolist()
            y_hat = model(x).view(-1, 24).tolist()
            true.extend(y)
            pred.extend(y_hat)

    pred = [[1 if i >= 0.5 else 0 for i in j] for j in pred]

    micro_f1 = f1_score(pred, true, average="micro")
    macro_f1 = f1_score(pred, true, average="macro")
    print("micro_f1: {:.4f}".format(micro_f1))
    print("micro_f1: {:.4f}".format(macro_f1))
    return micro_f1 * 0.4 + macro_f1 * 0.6


import heapq


def generate_test_result(config, dataset, model):
    with torch.no_grad():
        predict = []
        for idx, data in enumerate(dataset):
            x = [data[0].long(), data[1].long()]
            predict.extend(model(x).view(-1, 24).tolist())
    with open("submit.txt", "w", encoding="utf-8") as f:
        for i in range(len(predict)):
            line = []

            max_number = heapq.nlargest(3, predict[i])
            # max_index = []
            for t in max_number:
                index = predict[i].index(t)
                # max_index.append(index)
                line.append(config.id2label[index])

            # for j in range(len(predict[i])):
            #     if predict[i][j] >= 0.5:
            #         line.append(config.id2label[j])

            f.write(" ".join([str(i)] + line))
            f.write("\n")


In [6]:
import heapq

m = [
    0.016138209030032158,
    0.1027437224984169,
    0.007721974980086088,
    0.015482794493436813,
    0.03860097751021385,
    0.01681477390229702,
    0.0018505491316318512,
    0.2727064788341522,
    0.10750023275613785,
    0.10568385571241379,
    0.47227057814598083,
    0.059659115970134735,
    0.02180691622197628,
    0.7891892194747925,
    0.618352472782135,
    0.23794208467006683,
    0.011238433420658112,
    0.11568883806467056,
    0.07733272016048431,
    0.009712048806250095,
    0.011674805544316769,
    0.12361800670623779,
    0.00770289683714509,
    0.055381402373313904,
]
max_number = heapq.nlargest(3, m)
max_index = []
for t in max_number:
    index = m.index(t)
    max_index.append(index)
print(max_number)
print(max_index)


[0.7891892194747925, 0.618352472782135, 0.47227057814598083]
[13, 14, 10]


In [7]:
class Config:
    def __init__(self):
        self.pad_size = 500
        self.batch_size = 24
        self.epochs = 1
        self.PTM = "Yuang/unilm-base-chinese-news-sum"
        self.label_num = 24
        self.device = "cuda:0"
        self.lr = 5e-5

        label_dic = [
            "[微笑]",
            "[嘻嘻]",
            "[笑cry]",
            "[怒]",
            "[泪]",
            "[允悲]",
            "[憧憬]",
            "[doge]",
            "[并不简单]",
            "[思考]",
            "[费解]",
            "[吃惊]",
            "[拜拜]",
            "[吃瓜]",
            "[赞]",
            "[心]",
            "[伤心]",
            "[蜡烛]",
            "[给力]",
            "[威武]",
            "[跪了]",
            "[中国赞]",
            "[给你小心心]",
            "[酸]",
        ]

        self.id2label = {k: v for k, v in enumerate(label_dic)}  # 用于标签的部分
        self.label2id = {v: k for k, v in enumerate(label_dic)}


In [8]:
config = Config()
train_data = read_json("train.json")
valid_data = read_json("valid.json")
test_data = read_json("test.json")

tokenizer = RobertaTokenizer.from_pretrained(config.PTM)

train_dataloader = DataLoader(
    BaselineData(train_data, tokenizer, config), batch_size=config.batch_size
)
valid_dataloader = DataLoader(
    BaselineData(valid_data, tokenizer, config), batch_size=config.batch_size
)
test_dataloader = DataLoader(BaselineData(test_data, tokenizer, config), batch_size=1)


d:\Program Files\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected token

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
train_data[8]


In [ ]:
train_data[37]


In [ ]:
train_data[44]


In [ ]:
import re


# @ with : and @ without :
def clean_at(text):
    at_pattern = re.compile("@\S*:", re.S)
    text = re.sub(at_pattern, "", text)
    at_pattern = re.compile("@\S*", re.S)
    text = re.sub(at_pattern, "", text)
    return text.strip()


# Clear url in comments
def clean_url(text):
    sentences = text.split(" ")
    # 处理http://类链接
    url_pattern = re.compile(r"(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b", re.S)
    # 处理无http://类链接
    domain_pattern = re.compile(r"(\b)*(.*?)\.(com|cn)")
    if len(sentences) > 0:
        result = []
        for item in sentences:
            text = re.sub(url_pattern, "", item)
            text = re.sub(domain_pattern, "", text)
            result.append(text)
        return " ".join(result)
    else:
        return re.sub(url_pattern, "", sentences)


for i in range(len(train_data)):
    for j in range(len(train_data[i]["comments"])):
        text = train_data[i]["comments"][j]
        text_clean_at = clean_at(text)
        text_clean_url = clean_url(text_clean_at)
        train_data[i]["comments"][j] = text_clean_url

train_data[44]


In [ ]:
model = Model(config)  # .to(config.device)
optimizer = torch.optim.AdamW(model.parameters(), config.lr)


In [ ]:
best_model = train(config, train_dataloader, model, optimizer, valid_dataloader)
generate_test_result(config, test_dataloader, best_model)


In [ ]:
import sys

label_dic = [
    "[微笑]",
    "[嘻嘻]",
    "[笑cry]",
    "[怒]",
    "[泪]",
    "[允悲]",
    "[憧憬]",
    "[doge]",
    "[并不简单]",
    "[思考]",
    "[费解]",
    "[吃惊]",
    "[拜拜]",
    "[吃瓜]",
    "[赞]",
    "[心]",
    "[伤心]",
    "[蜡烛]",
    "[给力]",
    "[威武]",
    "[跪了]",
    "[中国赞]",
    "[给你小心心]",
    "[酸]",
]
id2label = {k: v for k, v in enumerate(label_dic)}  # 用于标签的部分
label2id = {v: k for k, v in enumerate(label_dic)}


def convert_label(fn_result):
    convert_label = []

    for line in open(fn_result, "r", encoding="utf-8"):
        labellist = line.strip().split(" ")[1:]

        onehot_label = [0] * 24
        for label in labellist:
            onehot_label[label2id[label]] = 1
        convert_label.append(onehot_label)
    return convert_label


def score_calculation(pred, true):
    macro_f1 = f1_score(pred, true, average="macro")
    return macro_f1


def main():
    """
    Generate classification_report from given pred and gold tsv files.
    """

    # Usage:
    # python evaluate PREDICTION_FILE GOLD_ANSWER_FILE
    # Example:
    # python evaluate pred.tsv gold.tsv
    print(len(sys.argv))
    print("The list of command line arguments:\n", sys.argv)

    if len(sys.argv) < 3:
        print("Please indicate the prediction and gold tsvs.")
        quit()
    else:
        pred_fn = sys.argv[1]
        gold_fn = sys.argv[2]

    print("Loading the datasets ...")
    pred_lbl = convert_label(pred_fn)
    gold_lbl = convert_label(gold_fn)

    print("Evaluating ...")
    try:
        macro_f1 = score_calculation(pred_lbl, gold_lbl)

        print("macro_f1: {:.4f}".format(macro_f1))
    except Exception as ex:
        print("error:", ex)
